In [51]:
# Region to install edge function. Note edge functions need to be created in us-east-1
# because reasons.
my_region = 'us-east-1'

In [52]:
# SDK setup
import boto3


cloudformation = boto3.client('cloudformation',region_name = my_region)

In [53]:
# Upload the zip and cf template
bucket_name = 'code97068'

In [54]:
%%bash -s "$bucket_name"
make
aws s3 cp edge.yml s3://$1
aws s3 cp edgefn.zip s3://$1

rm -f edgefn.zip
zip edgefn.zip edgefn.js
  adding: edgefn.js (deflated 40%)
upload: ./edge.yml to s3://code97068/edge.yml                 
upload: ./edgefn.zip to s3://code97068/edgefn.zip                   


In [55]:
def form_s3_url_prefix(region):
    prefix = ''
    if region == 'us-east-1':
        prefix = 'https://s3.amazonaws.com'
    else:
        prefix = 'https://s3-' + region + '.amazonaws.com'
    return prefix

In [56]:
edge_fn_stack = 'site-edge-fn-2'
bucket_base = form_s3_url_prefix(my_region) + '/' + bucket_name
print(bucket_base)

https://s3.amazonaws.com/code97068


In [57]:
response = cloudformation.create_stack(
    StackName=edge_fn_stack,
    TemplateURL= bucket_base + '/edge.yml',
    Parameters=[
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue':bucket_name
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM'
    ]
)

print(response)

{'StackId': 'arn:aws:cloudformation:us-east-1:427848627088:stack/site-edge-fn-2/036e1ee0-6824-11e8-8705-50d5cd265c36', 'ResponseMetadata': {'RequestId': '03667e8d-6824-11e8-99f3-751c0a7f86eb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '03667e8d-6824-11e8-99f3-751c0a7f86eb', 'content-type': 'text/xml', 'content-length': '384', 'date': 'Mon, 04 Jun 2018 18:20:51 GMT'}, 'RetryAttempts': 1}}


In [58]:
print('waiting on create of {}'.format(edge_fn_stack))
waiter = cloudformation.get_waiter('stack_create_complete')
waiter.wait(
    StackName=edge_fn_stack
)

print('stack created')

waiting on create of site-edge-fn-2
stack created


In [59]:
#Get the arn of the lambda function created above. We will need this to 
# set the edge function in our cloud front distribution.
response = cloudformation.describe_stacks(
    StackName=edge_fn_stack
)

print(response)

outputs = response['Stacks'][0]['Outputs']
lambda_arn = [d for d in outputs if d['OutputKey'] == 'LambdaArn'][0]['OutputValue']
print(lambda_arn)

{'Stacks': [{'StackId': 'arn:aws:cloudformation:us-east-1:427848627088:stack/site-edge-fn-2/036e1ee0-6824-11e8-8705-50d5cd265c36', 'StackName': 'site-edge-fn-2', 'Description': 'Edge function for exploring viewer request events\n', 'Parameters': [{'ParameterKey': 'CodeBucketName', 'ParameterValue': 'code97068'}], 'CreationTime': datetime.datetime(2018, 6, 4, 18, 20, 52, 122000, tzinfo=tzutc()), 'RollbackConfiguration': {}, 'StackStatus': 'CREATE_COMPLETE', 'DisableRollback': False, 'NotificationARNs': [], 'Capabilities': ['CAPABILITY_IAM'], 'Outputs': [{'OutputKey': 'LambdaArn', 'OutputValue': 'arn:aws:lambda:us-east-1:427848627088:function:site-edge-fn-2-EdgeProto-1CYVQ6WJIB3NP'}], 'Tags': [], 'EnableTerminationProtection': False}], 'ResponseMetadata': {'RequestId': '5f167a55-6824-11e8-80fb-991df5c98ef4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5f167a55-6824-11e8-80fb-991df5c98ef4', 'content-type': 'text/xml', 'content-length': '1544', 'date': 'Mon, 04 Jun 2018 18:

In [60]:
# Now we need to switch to region where the site distro stack was created
cdn_region = 'us-east-1'
cloudfront = boto3.client('cloudfront',region_name = cdn_region)
cloudformation = boto3.client('cloudformation',region_name = cdn_region)
cloudfront_stack_name = 'cf1'

In [61]:
response = cloudformation.describe_stack_resource(
    StackName=cloudfront_stack_name,
    LogicalResourceId='MyDistro'
)

distro_id = response['StackResourceDetail']['PhysicalResourceId']

print(distro_id)


E29589VNJARSJC


In [62]:
response = cloudfront.get_distribution_config(
    Id=distro_id
)

etag = response['ETag']
config = response['DistributionConfig']

print(config)

{'CallerReference': '689b5633-5d74-4783-95c2-b4b40023cca6', 'Aliases': {'Quantity': 0}, 'DefaultRootObject': '', 'Origins': {'Quantity': 1, 'Items': [{'Id': 'contentBucketOrigin', 'DomainName': 'foo97068.s3.amazonaws.com', 'OriginPath': '', 'CustomHeaders': {'Quantity': 0}, 'S3OriginConfig': {'OriginAccessIdentity': 'origin-access-identity/cloudfront/E1L6I1ZHVARFVG'}}]}, 'DefaultCacheBehavior': {'TargetOriginId': 'contentBucketOrigin', 'ForwardedValues': {'QueryString': False, 'Cookies': {'Forward': 'none'}, 'Headers': {'Quantity': 0}, 'QueryStringCacheKeys': {'Quantity': 0}}, 'TrustedSigners': {'Enabled': False, 'Quantity': 0}, 'ViewerProtocolPolicy': 'redirect-to-https', 'MinTTL': 0, 'AllowedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET'], 'CachedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET']}}, 'SmoothStreaming': False, 'DefaultTTL': 86400, 'MaxTTL': 31536000, 'Compress': False, 'LambdaFunctionAssociations': {'Quantity': 1, 'Items': [{'LambdaFunctionARN': 'arn:aws:lambda:us-e

In [63]:
print(config['DefaultCacheBehavior'])
for key in config['DefaultCacheBehavior']:
    print(key)

{'TargetOriginId': 'contentBucketOrigin', 'ForwardedValues': {'QueryString': False, 'Cookies': {'Forward': 'none'}, 'Headers': {'Quantity': 0}, 'QueryStringCacheKeys': {'Quantity': 0}}, 'TrustedSigners': {'Enabled': False, 'Quantity': 0}, 'ViewerProtocolPolicy': 'redirect-to-https', 'MinTTL': 0, 'AllowedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET'], 'CachedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET']}}, 'SmoothStreaming': False, 'DefaultTTL': 86400, 'MaxTTL': 31536000, 'Compress': False, 'LambdaFunctionAssociations': {'Quantity': 1, 'Items': [{'LambdaFunctionARN': 'arn:aws:lambda:us-east-1:427848627088:function:site-edge-fn-2-EdgeProto-EFSPERC39OAG:1', 'EventType': 'viewer-request'}]}, 'FieldLevelEncryptionId': ''}
TargetOriginId
ForwardedValues
TrustedSigners
ViewerProtocolPolicy
MinTTL
AllowedMethods
SmoothStreaming
DefaultTTL
MaxTTL
Compress
LambdaFunctionAssociations
FieldLevelEncryptionId


In [64]:
lambda_version = '1'
versioned_lambda_arn = lambda_arn + ':' + lambda_version
lambdaAssociation = {'Quantity': 1, 'Items':[{'LambdaFunctionARN':versioned_lambda_arn, 'EventType':'viewer-request'}]}
print(lambdaAssociation)

{'Quantity': 1, 'Items': [{'LambdaFunctionARN': 'arn:aws:lambda:us-east-1:427848627088:function:site-edge-fn-2-EdgeProto-1CYVQ6WJIB3NP:1', 'EventType': 'viewer-request'}]}


In [65]:
config['DefaultCacheBehavior']['LambdaFunctionAssociations'] = lambdaAssociation

In [66]:
print(config['DefaultCacheBehavior'])

{'TargetOriginId': 'contentBucketOrigin', 'ForwardedValues': {'QueryString': False, 'Cookies': {'Forward': 'none'}, 'Headers': {'Quantity': 0}, 'QueryStringCacheKeys': {'Quantity': 0}}, 'TrustedSigners': {'Enabled': False, 'Quantity': 0}, 'ViewerProtocolPolicy': 'redirect-to-https', 'MinTTL': 0, 'AllowedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET'], 'CachedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET']}}, 'SmoothStreaming': False, 'DefaultTTL': 86400, 'MaxTTL': 31536000, 'Compress': False, 'LambdaFunctionAssociations': {'Quantity': 1, 'Items': [{'LambdaFunctionARN': 'arn:aws:lambda:us-east-1:427848627088:function:site-edge-fn-2-EdgeProto-1CYVQ6WJIB3NP:1', 'EventType': 'viewer-request'}]}, 'FieldLevelEncryptionId': ''}


In [67]:
update_response = cloudfront.update_distribution(
    DistributionConfig=config,
    Id=distro_id,
    IfMatch=etag
)

print(update_response)

{'ResponseMetadata': {'RequestId': '62d0ea4f-6824-11e8-b1ad-a93d6b494041', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '62d0ea4f-6824-11e8-b1ad-a93d6b494041', 'etag': 'E3S2X6TQ5O9H1C', 'content-type': 'text/xml', 'content-length': '2975', 'vary': 'Accept-Encoding', 'date': 'Mon, 04 Jun 2018 18:23:31 GMT'}, 'RetryAttempts': 0}, 'ETag': 'E3S2X6TQ5O9H1C', 'Distribution': {'Id': 'E29589VNJARSJC', 'ARN': 'arn:aws:cloudfront::427848627088:distribution/E29589VNJARSJC', 'Status': 'InProgress', 'LastModifiedTime': datetime.datetime(2018, 6, 4, 18, 23, 32, 372000, tzinfo=tzutc()), 'InProgressInvalidationBatches': 0, 'DomainName': 'd37y2bas88eid3.cloudfront.net', 'ActiveTrustedSigners': {'Enabled': False, 'Quantity': 0}, 'DistributionConfig': {'CallerReference': '689b5633-5d74-4783-95c2-b4b40023cca6', 'Aliases': {'Quantity': 0}, 'DefaultRootObject': '', 'Origins': {'Quantity': 1, 'Items': [{'Id': 'contentBucketOrigin', 'DomainName': 'foo97068.s3.amazonaws.com', 'OriginPath': '', 'C